### 1. Temel Deformable Conv Bloğu (Offset + DeformConv2d)

Burada yapıyı şöyle düşün:

* offset_conv: Giriş feature map’ten offset üretiyor.

* dcn: DeformConv2d (torchvision.ops’ta var; yoksa mmcv benzeri kütüphaneden gelir.)

* Sonra normal Conv bloğu gibi BatchNorm + aktivasyon.

In [ ]:
import torch
import torch.nn as nn

try:
    from torchvision.ops import DeformConv2d
except ImportError:
    DeformConv2d = None

class DeformableConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels,
                 kernel_size=3, stride=1, padding=1, bias=False):
        super().__init__()

        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        # 1) Offset üreten konvolüsyon
        # Her konumda K*K tane grid noktası var,
        # her nokta için (dx, dy) → 2 * K * K kanal
        self.offset_conv = nn.Conv2d(
            in_channels,
            2 * kernel_size * kernel_size,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            bias=True,  # offset için bias genelde açık
        )

        if DeformConv2d is None:
            raise RuntimeError(
                "DeformConv2d bulunamadı. torchvision.ops veya DCN implementasyonu kurman gerekiyor."
            )

        # 2) Asıl deformable convolution
        self.dcn = DeformConv2d(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            bias=bias,
        )

        # 3) Normalleştirme + aktivasyon
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        # x: (B, C_in, H, W)

        # 1) Offset haritasını üret
        offset = self.offset_conv(x)
        # offset shape: (B, 2*K*K, H_out, W_out)

        # 2) Deformable Conv uygula
        out = self.dcn(x, offset)
        # out: (B, C_out, H_out, W_out)

        # 3) BN + ReLU
        out = self.bn(out)
        out = self.act(out)
        return out


if __name__ == "__main__":
    x = torch.randn(1, 64, 128, 128)
    block = DeformableConvBlock(64, 128, kernel_size=3, stride=1, padding=1)
    y = block(x)
    print("Input:", x.shape)
    print("Output:", y.shape)


Input: torch.Size([1, 64, 128, 128])
Output: torch.Size([1, 128, 128, 128])


### Bu yapıda ne oluyor?

**offset_conv:**

* Input: (B, C_in, H, W)

* Output: (B, 2*K*K, H_out, W_out) → her spatial konumda grid’in nereye kayacağı çıkıyor.

**dcn(x, offset):**

### DCN içerde şunu yapıyor:

* Her konum için sabit grid + offset → yeni koordinatlar

* Bilinear interpolasyon ile bu koordinatlardan değer çekiyor

* Ağırlıklarla çarpıp topluyor (normal conv gibi)

**Senin için kritik olan:**

>Değişen tek şey, örnekleme noktalarının koordinatları; geri kalan mantık Conv ile aynı.

----
----
## 2. Modulated DCN (DCNv2 Mantığı)

* Burada offset + bir de modulation mask var. Maske her grid noktasının önemini ayarlıyor.

* Ek olarak mask_conv var.

* Mask kanal sayısı = K*K (her grid noktası için 1 skaler ağırlık).

In [3]:
class ModulatedDeformableConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels,
                 kernel_size=3, stride=1, padding=1, bias=False):
        super().__init__()

        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        # Offset: 2 * K * K
        self.offset_conv = nn.Conv2d(
            in_channels,
            2 * kernel_size * kernel_size,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        )

        # Mask: K * K
        self.mask_conv = nn.Conv2d(
            in_channels,
            kernel_size * kernel_size,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        )

        if DeformConv2d is None:
            raise RuntimeError("DeformConv2d bulunamadı.")

        # torchvision.ops.DeformConv2d modülasyon maskesini doğrudan desteklemiyor olabilir.
        # DCNv2 implementasyonlarında genelde farklı bir sınıf var (ModulatedDeformConv2d).
        # Burada konsepti göstermek için mask'i dışarıdan uygular gibi düşüneceğiz.
        self.dcn = DeformConv2d(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            bias=bias,
        )

        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        # 1) Offset ve mask üret
        offset = self.offset_conv(x)                       # (B, 2*K*K, H, W)
        mask = self.mask_conv(x)                          # (B, K*K, H, W)
        mask = torch.sigmoid(mask)                        # [0,1] aralığına sık

        # 2) DCN çağrısında bazı implementasyonlar mask'i ayrıca alır:
        # out = self.dcn(x, offset, mask)
        # Fakat torchvision.ops.DeformConv2d bunu desteklemiyorsa,
        # DCNv2 için özel implementasyon kullanmak gerekir.
        # Burada sadece kavramsal gösteriyoruz.
        out = self.dcn(x, offset)  # mask'siz basit kullanım

        # 3) Aktivasyon katmanları
        out = self.bn(out)
        out = self.act(out)
        return out


## 3. İç Mantığı Anlamak İçin: grid_sample ile Basit DCN Simülasyonu

* Gerçekte DeformConv2d içinde olan şeyin bir benzerini, F.grid_sample ile kendimiz yazabiliriz.
* Bu hem offset fikrini hem de koordinat sistemini çok netleştirir.

Burada:

* Giriş: (B, C, H, W)

* Base grid: her piksel için normal (sabit) koordinat

* Offset: bu grid’e eklenen kayma

- grid_sample: mis gibi deformable sampling.

In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn


class ToyDeformableSampler(nn.Module):
    """
    Bu sınıf tam DCN değil, sadece 'offset + sampling' mantığını anlamak için.
    Fikir:
      - Bir offset haritası var: (B, H, W, 2) → (dx, dy)
      - Bu offset'i normal bir grid'e ekleyip F.grid_sample ile örnekliyoruz.
    """
    def __init__(self):
        super().__init__()

    def forward(self, x, offset):
        """
        x:      (B, C, H, W)
        offset: (B, H, W, 2)  # her piksel için (dx, dy)
        """
        B, C, H, W = x.shape

        # 1) Normalized base grid oluştur: [-1,1] aralığında
        #    grid_x: soldan sağa -1 → 1
        #    grid_y: yukarıdan aşağı -1 → 1
        grid_y, grid_x = torch.meshgrid(
            torch.linspace(-1, 1, H, device=x.device),
            torch.linspace(-1, 1, W, device=x.device),
            indexing="ij"
        )
        base_grid = torch.stack((grid_x, grid_y), dim=-1)  # (H, W, 2)
        base_grid = base_grid.unsqueeze(0).repeat(B, 1, 1, 1)  # (B, H, W, 2)

        # 2) Offsetleri bu base_grid'e ekle
        #    offset de normalized koordinat sisteminde olmalı
        #    (gerçekte DCN integer piksel düzleminde çalışıyor; burada görselleştirme için normalleştiriyoruz.)
        deformed_grid = base_grid + offset  # (B, H, W, 2)

        # 3) grid_sample ile yeni grid'e göre x'ten değer çek
        # mode='bilinear', padding_mode='zeros' vs. ayarlanabilir.
        sampled = F.grid_sample(
            x,
            deformed_grid,
            mode="bilinear",
            padding_mode="border",
            align_corners=True,
        )
        # sampled: (B, C, H, W)

        return sampled


if __name__ == "__main__":
    B, C, H, W = 1, 3, 32, 32
    x = torch.randn(B, C, H, W)

    # Basit bir offset haritası üretelim: tüm görüntüyü hafif sağa kaydırıyoruz
    offset = torch.zeros(B, H, W, 2)
    offset[..., 0] = 0.1  # x yönünde kayma
    offset[..., 1] = 0.0  # y yönü sabit

    sampler = ToyDeformableSampler()
    y = sampler(x, offset)
    print("Input:", x.shape)
    print("Output:", y.shape)


Input: torch.Size([1, 3, 32, 32])
Output: torch.Size([1, 3, 32, 32])


## 4. Backbone’a Entegre Ederken Kod Yapısı Nasıl Olur?

* Mesela diyelim ki, kendi CNN’inde şöyle bir blok var:

In [3]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.bn1   = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.bn2   = nn.BatchNorm2d(out_channels)
        self.act   = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x
        out = self.act(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        out = self.act(out)
        return out


**Bunu DCN’le değiştirmek istersek:**

In [4]:
class DeformableBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = DeformableConvBlock(
            in_channels,
            out_channels,
            kernel_size=3,
            stride=1,
            padding=1,
        )  # DCN blok
        self.conv2 = DeformableConvBlock(
            out_channels,
            out_channels,
            kernel_size=3,
            stride=1,
            padding=1,
        )

        self.proj = None
        if in_channels != out_channels:
            # Kanal sayısı uymazsa, identity'yi projeksiyonla çevir.
            self.proj = nn.Conv2d(in_channels, out_channels, 1, bias=False)

        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x if self.proj is None else self.proj(x)

        out = self.conv1(x)
        out = self.conv2(out)

        out = out + identity
        out = self.act(out)
        return out


Burada çok net:

* Sadece nn.Conv2d yerine DeformableConvBlock koyuyoruz.

* Geri kalanı birebir residual block mantığı.

---
----

#  Neden 2 * kernel_size * kernel_size yazdık?

Çünkü deformable conv, her kernel örnekleme noktası için 2 tane offset ister:

* Δx (sağa-sola kayma)

* Δy (yukarı-aşağı kayma)

* Kernel boyutu K×K ise toplam K*K adet örnekleme noktası vardır.

Her nokta için 2 değer (dx, dy) gerektiği için:

offset kanal sayısı =  2 × (K×K)

Örnekler:

* K=3 → K*K = 9 nokta var

  → 2*9 = 18 kanal offset çıkar.

* K=5 → 25 nokta

→ 2*25 = 50 kanal offset çıkar.

### Bu kanallar neyi temsil ediyor?

Offset conv’un çıktısı şu şekildedir:
```python
offset.shape = (B, 2*K*K, H_out, W_out)
```

Bu 2*K*K kanal aslında şunun gibi düşün:

* İlk K*K kanal: tüm noktaların Δx değerleri

* Son K*K kanal: tüm noktaların Δy değerleri

Yani her konum (i,j) için model şunu öğreniyor:

* “3×3 kernelin 9 noktasını, bu konumda şu kadar sağa-sola ve yukarı-aşağı kaydır.”

Bu yüzden 2*kernel*kernel zorunlu. Başka sayı verirsen DCN katmanı offset’i doğru okuyamaz.


----
----